In [1]:
import numpy as np

import astropy.table as astropyTable
from astropy.io.votable import parse

In [2]:
def read_pointings(filePath="",maxNight=32, keepOnlyImageType=True):
    """Will read VOT metadata tables for series of VOT table filenames given
    in night_{index}.vot format, where index is incremented index from 0 to
    maxNight and stack them into a singular table. 

    Columns 'archive_file' and 'prodtype' renamed to 'filename' and 'product'.
    Approx. 15 columns are removed as they are not very useful to us.
    Columns 'visit_id', 'reprocessing', 'ingest_folder' and  'imdiff_folder' are
    added. Visit id and reprocessing are essentially just more manageable form
    of existing columns while ingest and imdiff are set to None by default and
    will not meaningfully represent any information untill the pointings have
    been grouped.

    Parameters
    ----------
    maxNight : int
        index marking the last file in series of night_N.vot files that will be
        read.
    keepOnlyImageType : bool
        if True only the products of type "image" will be kept. True by default.
    """
    allNights = []
    for i in range(1, maxNight):
        filename = filePath+"night_{0}.vot".format(i)
        votableFile = parse(filename)
        # also the only table availible
        votable = votableFile.get_first_table() 

        table = votable.to_table()
        table.remove_columns(["md5sum", "filesize", "depth", "seeing", "dtpi",
                              "instrument", "telescope", "surveyid", "dtpropid",
                              "reference", "filter", "proctype", "obsmode",
                              "obstype"])
        col = astropyTable.Column( [i]*len(table) , name="survey_night")
        table.add_column(col)
        allNights.append(table)

    allNights = astropyTable.vstack(allNights)

    allNights.rename_column("archive_file", "filename")
    allNights.rename_column("prodtype", "product")

    if keepOnlyImageType:
        allNights = allNights[allNights["product"] == b"image"]

    visit_ids = map(lambda x: x[-14:-8], allNights["dtacqnam"])
    col = astropyTable.Column( list(visit_ids) , name="visit_id")
    allNights.add_column(col)

    # reprocessing version is hidden in the last section of the full filename
    # so we cut it out into a standalone column to make it easier to work with
    reprocessing_id = map(lambda x:
                          int(x.split(b".")[0][-1:]), allNights["filename"])
    col = astropyTable.Column( list(reprocessing_id), name="reprocessing")
    allNights.add_column(col)

    col = astropyTable.Column( [None]*len(allNights) , name="ingest_folder")
    allNights.add_column(col)

    col = astropyTable.Column( [None]*len(allNights) , name="imdiff_folder")
    allNights.add_column(col)

    # we sort the nights provisionaly just to match Hayden's order, otherwise
    # not required
    allNights.sort(["ra", "dec"])

    return allNights

In [5]:
pointings=read_pointings('VOTMetadata/')

In [9]:
group = pointings
allVisitIds = group["visit_id"].data.data.astype(np.int)
uniqueVisitIds  = np.unique(allVisitIds)
# For each unique visit in a group we find the max reprocessing version
maxReprocessing = {u: max(group[allVisitIds==u]["reprocessing"]) \
                   for u in uniqueVisitIds}

In [15]:
maxReprocessing.values()

dict_values([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 